In [1]:
using GAP
using JSON3


LoadError: ArgumentError: Package GAP not found in current path.
- Run `import Pkg; Pkg.add("GAP")` to install the GAP package.

In [ ]:
# json ファイルの読み取り
config_raw = JSON3.read(read("config.json", String))

"""
JSONのサイクルリストからGAPの置換オブジェクトを生成する
例: [[1,3,8,6], [2,5,7,4]]  =>  (1,3,8,6)(2,5,7,4)
"""
function reconstruct_gap_perm(cycles_list)
    # 各サイクルを "(1,3,8,6)" のような文字列に変換し、結合する
    # GAP.evalstr は文字列をGAPの式として評価してオブジェクトを返します
    gap_str = join(["(" * join(c, ",") * ")" for c in cycles_list], "")

    # 恒等置換（どこも動かない）の場合は空文字列になるので、その場合は "()" を渡す
    if gap_str == ""
        return @gap ()
    end

    return GAP.evalstr(gap_str)
end


In [ ]:
# 2. 各面を再構築して辞書に格納
# これで Core 定義を外部ファイルから注入できます
moves = Dict()
for (face_name, cycles) in config_raw.definitions
    moves[string(face_name)] = reconstruct_gap_perm(cycles)
end


In [ ]:
# 3. 以前の Core 定義のように変数へ割り当てる
U, L, F, R, B, D = moves["U"], moves["L"], moves["F"], moves["R"], moves["B"], moves["D"]

cube = (@gap Group)(U, L, F, R, B, D)
words = @gap ["U", "L", "F", "R", "B", "D"]
free = (@gap FreeGroup)(words)
hom = (@gap GroupHomomorphismByImages)(free, cube, (@gap GeneratorsOfGroup)(free), (@gap GeneratorsOfGroup)(cube))

scramble_array = [rand(["U", "L", "F", "R", "B", "D"]) for _ in 1:20]
scramble_str = join(scramble_array, " ")

σ = @gap ()
for s in scramble_array
    idx = findfirst(==(s), ["U", "L", "F", "R", "B", "D"])
    global σ *= (@gap GeneratorsOfGroup)(cube)[idx]
end

word_raw = string((@gap PreImagesRepresentative)(hom, σ))

function gap_to_lsystem(word_str)
    res = replace(word_str, "*" => " ")
    res = replace(res, "^-1" => "-")
    res = replace(res, "^-2" => "++")
    res = replace(res, "^2" => "++")
    return join([occursin(r"[\+\-]", m) ? m : m * "+" for m in split(res)], " ")
end

data = Dict(
    "scramble" => gap_to_lsystem(scramble_str),
    "solution" => gap_to_lsystem(word_raw)
)
open("solve.json", "w") do f
    JSON3.pretty(f, data)
end
